In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("ColabSparkSession") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/16 15:04:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

(spark
 .read
 .option("header","true")
 .csv("spark-data/ratings.csv")
 .coalesce(3)
 .write
 .mode("overwrite")
 .bucketBy(10,"movieId")
 .saveAsTable("rating_bucketed")
)

In [3]:
spark.sql("Describe formatted rating_bucketed").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|              userId|              string|   NULL|
|             movieId|              string|   NULL|
|              rating|              string|   NULL|
|           timestamp|              string|   NULL|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|             Catalog|       spark_catalog|       |
|            Database|             default|       |
|               Table|     rating_bucketed|       |
|        Created Time|Wed Jul 16 15:04:...|       |
|         Last Access|             UNKNOWN|       |
|          Created By|         Spark 4.0.0|       |
|                Type|             MANAGED|       |
|            Provider|             parquet|       |
|         Num Buckets|                  10|       |
|      Bucket Columns|         [`movieId`]|       |
|        Sor

In [4]:
(spark
 .read
 .table("rating_bucketed")
 .groupBy("movieId")
 .count()
 .write
 .mode("overwrite")
 .csv("spark-data/output")
)

In [5]:
(spark
 .read
 .parquet("spark-data/rating-huge")
 .groupBy("movieId")
 .count()
 .write
 .mode("overwrite")
 .csv("spark-data/output")
)

25/07/16 15:05:23 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: spark-data/rating-huge.
java.io.FileNotFoundException: File spark-data/rating-huge does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.Resolve

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/shivam/Downloads/Spark/Spark_Optimization_new/Day2/spark-data/rating-huge. SQLSTATE: 42K03